In [1]:
# This script runs daily at 6pm
# Scheduled using Azure Logic Apps
import requests
from bs4 import BeautifulSoup
import time
import csv
import pyodbc
import pandas as pd

In [2]:
# MarketWatch.com shows instruments for Sri Lanka in the following URLs as two pages
# https://www.marketwatch.com/tools/markets/stocks/country/sri-lanka
# https://www.marketwatch.com/tools/markets/stocks/country/sri-lanka/2

# Latter part of the instrument urls are added to a list (ex:['/investing/Stock/ABAN.N0000?countryCode=LK',
#                                            '/investing/Stock/AFSL.N0000?countryCode=LK', etc...]

def get_partial_instrument_links_from_page(url, maxlinks):
    list_of_instrument_urls_in_page = []
    items = requests.get(url)
    soup_it = BeautifulSoup(items.content,
                            'html5lib')

    while len(list_of_instrument_urls_in_page) < maxlinks:
        for links_page1 in soup_it.find_all('a'):
            link = links_page1.get('href')

            if link:
                if '/investing/Stock/' in link:
                    list_of_instrument_urls_in_page.append(link)
    return list_of_instrument_urls_in_page

In [3]:
def add_instrument_data_to_dic(dic, name_of_mkt_data, value_of_mkt_data):
    if name_of_mkt_data not in dic:
        dic[name_of_mkt_data] = []
    dic[name_of_mkt_data].append(value_of_mkt_data)


In [4]:
url_page1 = 'https://www.marketwatch.com/tools/markets/stocks/country/sri-lanka'
url_page2 = 'https://www.marketwatch.com/tools/markets/stocks/country/sri-lanka/2'

list_of_instruments_page1 = get_partial_instrument_links_from_page(url_page1, 150)
list_of_instruments_page2 = get_partial_instrument_links_from_page(url_page2, 124)

print("URLs from page 1: " + str(len(list_of_instruments_page1)))
print("URLs from page 2: " + str(len(list_of_instruments_page2)))

List_of_instruments = list_of_instruments_page1 + list_of_instruments_page2

URLs from page 1: 150
URLs from page 2: 130


In [5]:
# # # # # # # # # # # # #

List_of_dic = []

instrument_data = {}

# The following for loop will:
#           1. Create the instrument urls
#           2. Scrape the webpage to get the needed information from the urls
#           3. Add them to instrument_data dictionary (one dictionary to each instrument)
#           4. Append the dictionaries to a list

for instrument in List_of_instruments:
    url = 'https://www.marketwatch.com' + instrument

    r = requests.get(url)

    soup = BeautifulSoup(r.content,
                         'html5lib')
    soup.prettify()

    data_for_dic = soup.findAll("li", {"class": "kv__item"})

    for row in data_for_dic:
        tag = row.find_all("small", {"class": "label"})
        value = row.find_all("span", {"class": "primary"})

        add_instrument_data_to_dic(instrument_data, tag[0].string, value[0].string)

    instrument_data['Instrument'] = instrument[17:]

    List_of_dic.append(instrument_data)

    print(len(List_of_dic))

    time.sleep(1)


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [9]:
# Writing a csv file using the data

csv_columns = ['Open', 'Day Range', '52 Week Range', 'Market Cap',
               'Shares Outstanding', 'Public Float', 'Beta', 'Rev. per Employee', 'P/E Ratio', 'EPS',
               'Yield', 'Dividend', 'Ex-Dividend Date', 'Short Interest', '% of Float Shorted',
               'Average Volume', 'Instrument']

csv_file = "data/Market_key_data.csv"

try:
    with open(csv_file, 'w', encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=csv_columns)
        writer.writeheader()
        for data in List_of_dic:
            writer.writerow(data)
except IOError:
    print("I/O error")

In [10]:
# Copy the data into Market_key_data_sanitized.csv after removing unnecessary characters and words in
#                                                                                        Market_key_data.csv

with open("data/Market_key_data.csv", "r") as infile, open("Market_key_data_sanitized.csv", "w") as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile)
    conversion = set("]['")
    for row in reader:
        newrow = ["".join("" if c in conversion else c for c in entry) for entry in row]
        for index, item in enumerate(newrow):
            newrow[index] = newrow[index].replace("රු.", "")
            newrow[index] = newrow[index].replace("?countryCode=LK", "")
            newrow[index] = newrow[index].replace("N/A", "0")
            newrow[index] = newrow[index].replace(',', '')
        writer.writerow(newrow)


In [11]:
# Feed the data into a pandas dataframe
# Edit the column names to match the column names of the database
# change the data types to match the database accordingly

df = pd.read_csv("/Users/Dell/python files/Market_key_data_sanitized.csv")
df.columns = df.columns.str.replace('%', '')
df.columns = df.columns.str.replace('[/,-]', ' ')
df.columns = df.columns.str.replace(' ', '_')

df.rename(columns={'52_Week_Range': 'my_52_Week_Range', '_of_Float_Shorted': 'of_Float_Shorted',
                   'Rev._per_Employee': 'Rev_per_Employee'}, inplace=True)

df[['Open', 'Beta', 'P_E_Ratio', 'EPS', 'Dividend', 'Short_Interest', 'of_Float_Shorted']] = df[
    ['Open', 'Beta', 'P_E_Ratio', 'EPS', 'Dividend', 'Short_Interest', 'of_Float_Shorted']].astype(str)
print(df.dtypes)

Open                  object
Day_Range             object
my_52_Week_Range      object
Market_Cap            object
Shares_Outstanding    object
Public_Float          object
Beta                  object
Rev_per_Employee      object
P_E_Ratio             object
EPS                   object
Yield                 object
Dividend              object
Ex_Dividend_Date      object
Short_Interest        object
of_Float_Shorted      object
Average_Volume        object
Instrument            object
dtype: object


C:\Users\Dell\AppData\Local\Temp/ipykernel_19612/2767464830.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  df.columns = df.columns.str.replace('[/,-]', ' ')


In [23]:
# Connection between the python script and the database is now been made
# The data is overwritten to the database
# Overwriting is done because the interest is about the data on a particular day for each instrument.
#                                                                                      (ex: EPS, Market_cap)
import pyodbc as db
conn = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};Server=localhost;Database=CSEDB;Uid=sa;Pwd=123")

conn.setdecoding(db.SQL_CHAR, encoding='latin-1')
conn.setencoding('latin-1')

cursor = conn.cursor()

#cursor.execute('''
#               DELETE FROM CSEDB.dbo.Market_data_test
#               ''')


#for row in df.itertuples(index=True, name='Pandas'):
#   cursor.execute('INSERT INTO Market_data("Open", Day_Range, "_52_Week_Range", Market_Cap, Shares_Outstanding, '
#                   'Public_Float, Beta, Rev_per_Employee, P_E_Ratio, EPS, Yield, Dividend, Ex_Dividend_Date, '
#                   'Short_Interest, of_Float_Shorted, Average_Volume, Instrument) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,'
#                   '?,?,?)',
#                   row.Open, row.Day_Range, row.my_52_Week_Range, row.Market_Cap, row.Shares_Outstanding,
#                   row.Public_Float, row.Beta, row.Rev_per_Employee, row.P_E_Ratio, row.EPS, row.Yield, row.Dividend,
#                   row.Ex_Dividend_Date, row.Short_Interest, row.of_Float_Shorted, row.Average_Volume, row.Instrument)
    
for row in df.itertuples(index=True, name='Pandas'):
    cursor.execute("""INSERT INTO Market_data("Open", Day_Range, "_52_Week_Range", Market_Cap, Shares_Outstanding,
                   Public_Float, Beta, Rev_per_Employee, P_E_Ratio, EPS, Yield, Dividend, Ex_Dividend_Date,
                   Short_Interest, of_Float_Shorted, Average_Volume, Instrument) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,
                   ?,?,?)""",
                   row.Open, row.Day_Range, row.my_52_Week_Range, row.Market_Cap, row.Shares_Outstanding,
                   row.Public_Float, row.Beta, row.Rev_per_Employee, row.P_E_Ratio, row.EPS, row.Yield, row.Dividend,
                   row.Ex_Dividend_Date, row.Short_Interest, row.of_Float_Shorted, row.Average_Volume, row.Instrument)

conn.commit()
cursor.close()
conn.close()


UnicodeEncodeError: 'latin-1' codec can't encode characters in position 0-1: ordinal not in range(256)

In [ ]:
for row in df.itertuples(index=True, name='Pandas'):
    cursor.execute('INSERT INTO Market_data("Open", Day_Range, "_52_Week_Range", Market_Cap, Shares_Outstanding, '
                   'Public_Float, Beta, Rev_per_Employee, P_E_Ratio, EPS, Yield, Dividend, Ex_Dividend_Date, '
                   'Short_Interest, of_Float_Shorted, Average_Volume, Instrument) values(?,?,?,?,?,?,?,?,?,?,?,?,?,?,'
                   '?,?,?)',
                   row.Open, row.Day_Range, row.my_52_Week_Range, row.Market_Cap, row.Shares_Outstanding,
                   row.Public_Float, row.Beta, row.Rev_per_Employee, row.P_E_Ratio, row.EPS, row.Yield, row.Dividend,
                   row.Ex_Dividend_Date, row.Short_Interest, row.of_Float_Shorted, row.Average_Volume, row.Instrument)